In [11]:
! pip install pandas snowflake-connector-python snowflake-sqlalchemy

     ---------------------------------------- 0.0/72.3 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/72.3 kB ? eta -:--:--
     ---------------- --------------------- 30.7/72.3 kB 262.6 kB/s eta 0:00:01
     ---------------- --------------------- 30.7/72.3 kB 262.6 kB/s eta 0:00:01
     ---------------- --------------------- 30.7/72.3 kB 262.6 kB/s eta 0:00:01
     ---------------- --------------------- 30.7/72.3 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 72.3/72.3 kB 248.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/948.5 kB ? eta -:--:--
   --- ------------------------------------ 81.9/948.5 kB 2.3 MB/s eta 0:00:01
   ---------- ----------------------------- 245.8/948.5 kB 3.8 MB/s eta 0:00:01
   ------------ --------------------------- 307.2/948.5 kB 3.2 MB/s eta 0:00:01
   ------------ --------------------------- 307.2/948.5 kB 3.2 MB/s eta 0:00:01
   ----------------- ---------------------- 409.6/948.5 kB 1.8 MB


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import snowflake.connector
from snowflake.sqlalchemy import URL

In [2]:
conn = create_engine(URL(
    user='PALASHHERE',
    password='SureshRaina3004@',
    account='foa54253.east-us-2.azure',  
    warehouse='COMPUTE_WH',
    database='HEALTH_DATA',
    schema='CHRONIC_DISEASE'
))


In [3]:
query = "SELECT * FROM RAW_INDICATORS"
df = pd.read_sql(query, conn)

In [4]:
df.shape

(309215, 34)

In [32]:
df.dropna(axis=1, how='all', inplace=True)

In [33]:
df.columns

Index(['yearstart', 'yearend', 'locationabbr', 'locationdesc', 'datasource',
       'topic', 'question', 'datavalueunit', 'datavaluetype', 'datavalue',
       'datavaluealt', 'datavaluefootnotesymbol', 'datavaluefootnote',
       'lowconfidencelimit', 'highconfidencelimit', 'stratificationcategory1',
       'stratification1', 'geolocation', 'locationid', 'topicid', 'questionid',
       'datavaluetypeid', 'stratificationcategoryid1', 'stratificationid1'],
      dtype='object')

In [34]:
df = df[df["datavalue"].notnull()]


In [35]:
# Convert 'datavalue' to numeric (just in case)
df["datavalue"] = pd.to_numeric(df["datavalue"], errors='coerce')


In [36]:
df.rename(columns={
    "yearstart": "year",
    "locationdesc": "state",
    "locationabbr": "state_code",
    "datavalue": "value",
    "datavaluetype": "value_type",
    "lowconfidencelimit": "ci_lower",
    "highconfidencelimit": "ci_upper",
    "stratification1": "group",
    "stratificationcategory1": "group_category"
}, inplace=True)

In [37]:
df = df[[
    "year", "state_code", "state", "topic", "question", "value", "value_type",
    "ci_lower", "ci_upper", "datavalueunit", "geolocation",
    "group_category", "group",
    "locationid", "topicid", "questionid", "datavaluetypeid", "stratificationcategoryid1", "stratificationid1"
]]

In [39]:
df.rename(columns={'group':'group_name'}, inplace=True)

### Sampling only 200000 records because of the upload limit of SQLAlchemy and Snowflake

In [44]:
df  = df.head(200000)

### Data will be uploaded in one go

In [45]:
df.to_sql('clean_indicators', con=conn, if_exists='replace', index=False)

print("Full cleaned dataset uploaded for deep analysis!")

Full cleaned dataset uploaded for deep analysis!
